In [1]:
import sys
import scipy.stats
from scipy.signal import coherence
from scipy.cluster.hierarchy import fcluster
from sklearn import metrics
from sklearn.linear_model import LinearRegression

from trajectory_process import traj_process

import numpy as np
from scipy.cluster import hierarchy
from sklearn import cluster
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib notebook
import glob
import re  # Regular expression operations
import pandas as pd
from scipy import signal
import mne
from tqdm import tqdm
import os
from os.path import exists
import pickle
import statsmodels.stats.api as sms

base_dir = '/home/fu/Data/ephys/'
# base_dir = 'D:\ephys\'

import utils
import ephys
import plotting

plt.rcParams["axes.labelsize"] = 12
plt.rcParams["font.size"] = 7
# plt.rcParams["font.family"] = "Arial"

mBWfus009 = {
    'arena_0219': '2021-02-19_mBWfus009_arena_ephys',
    'ezm_0219': '2021-02-19_mBWfus009_EZM_ephys',
    'oft_0219': '2021-02-19_mBWfus009_OF_ephys',

    'arena_0226': '2021-02-26_mBWfus009_arena_ephys',
    'ezm_0226': '2021-02-26_mBWfus009_EZM_ephys',
    'oft_0226': '2021-02-26_mBWfus009_OF_ephys',

    'arena_0305_bef': '2021-03-05_mBWfus009_before_arena_ephys',
    'cage_0305': '2021-03-05_mBWfus009_cage_arena_ephys',
    'arena_0305_aft': '2021-03-05_mBWfus009_after_arena_ephys',
    'ezm_0305': '2021-03-05_mBWfus009_EZM_ephys',
    'oft_0305': '2021-03-05_mBWfus009_OF_ephys',

    'arena_0325': '2021-03-25_mBWfus009_arena_ephys',
    'epm_0325': '2021-03-25_mBWfus009_EPM_ephys'
}

mBWfus008 = {
    'arena_0219': '2021-02-19_mBWfus008_arena_ephys',
    'ezm_0219': '2021-02-19_mBWfus008_EZM_ephys',
    'oft_0219': '2021-02-19_mBWfus008_OF_ephys',

    'arena_0226': '2021-02-26_mBWfus008_arena_ephys',
    'ezm_0226': '2021-02-26_mBWfus008_EZM_ephys',
    'oft_0226': '2021-02-26_mBWfus008_OF_ephys',

    'arena_0305_bef': '2021-03-05_mBWfus008_before_arena_ephys',
    'cage_0305': '2021-03-05_mBWfus008_cage_arena_ephys',
    'arena_0305_aft': '2021-03-05_mBWfus008_after_arena_ephys',
    'ezm_0305': '2021-03-05_mBWfus008_EZM_ephys',
    'oft_0305': '2021-03-05_mBWfus008_OF_ephys',
}

mBWfus010 = {
    'arena_0219': '2021-02-19_mBWfus010_arena_ephys',
    'ezm_0219': '2021-02-19_mBWfus010_EZM_ephys',
    'oft_0219': '2021-02-19_mBWfus010_OF_ephys',

    'arena_0301_aft': '2021-03-01_mBWfus010_arena_ephys_after',
    'arena_0301_bef': '2021-03-01_mBWfus010_arena_ephys_before',
    'cage_0301': '2021-03-01_mBWfus010_cage_ephys',
    'oft_0301': '2021-03-01_mBWfus010_OF_ephys',
    'ezm_0301': '2021-03-01_mBWfus010_EZM_ephys',

    'arena_0307_bef': '2021-03-07_mBWfus010_after_arena_ephys',
    'cage_0307': '2021-03-07_mBWfus010_cage_arena_ephys',
    'arena_0307_aft': '2021-03-07_mBWfus010_after_arena_ephys',
    'ezm_0307': '2021-03-07_mBWfus010_EZM_ephys',
    'oft_0307': '2021-03-07_mBWfus010_OF_ephys',
}

mBWfus011 = {
    'arena_0226': '2021-02-26_mBWfus011_arena_ephys',
    'ezm_0226': '2021-02-26_mBWfus011_EZM_ephys',
    'oft_0226': '2021-02-26_mBWfus011_OF_ephys',

    'arena_0305_aft': '2021-03-05_mBWfus011_after_arena_ephys',
    'cage_0305': '2021-03-05_mBWfus011_cage_arena_ephys',
    'arena_0305_bef': '2021-03-05_mBWfus011_before_arena_ephys',
    'oft_0305': '2021-03-05_mBWfus011_OF_ephys',
    'ezm_0305': '2021-03-05_mBWfus011_EZM_ephys',

    'arena_0313_bef': '2021-03-13_mBWfus011_before_arena_ephys',
    'cage_0313': '2021-03-13_mBWfus011_cage_arena_ephys',
    'arena_0313_aft': '2021-03-13_mBWfus011_after_arena_ephys',
    'ezm_0313': '2021-03-13_mBWfus011_EZM_ephys',
    'oft_0313': '2021-03-13_mBWfus011_OF_ephys',
}

mBWfus012 = {
    'arena_0226': '2021-02-26_mBWfus012_arena_ephys',
    'ezm_0226': '2021-02-26_mBWfus012_EZM_ephys',
    'oft_0226': '2021-02-26_mBWfus012_OF_ephys',
}

In [ ]:
np.random.seed(42)
animal = mBWfus011
session = 'arena_0313_aft'
behavior_trigger = 4.62 # time when LED is turned off
start_time = 0.0 # time when animal is put onto maze
savedir='./results/011_0313_arena_aft/'

In [ ]:
f_behavior = 50
f_ephys = 500
behavior_window_duration = 900
ephys_window_duration = 1000
events = traj_process(base_dir, animal[session], behavior='arena', start_time=start_time, duration=behavior_window_duration)

In [ ]:
dataset = ephys.load_data(base_dir, animal[session])

power_mpfc = ephys.column_by_pad(ephys.get_power(dataset, 'mpfc', 'theta'))
power_vhipp = ephys.column_by_pad(ephys.get_power(dataset, 'vhipp', 'theta'))

mpfc_pads = np.array(power_mpfc.columns)
vhipp_pads = np.array(power_vhipp.columns)

totlen = len(mpfc_pads) + len(vhipp_pads)
power_corrs = np.zeros((totlen, totlen))
for i in range(totlen):
    for j in range(totlen):
        if i < len(mpfc_pads):
            siga = power_mpfc[mpfc_pads[i]]
        else:
            siga = power_vhipp[vhipp_pads[i-len(mpfc_pads)]]
        if j < len(mpfc_pads):
            sigb = power_mpfc[mpfc_pads[j]]
        else:
            sigb = power_vhipp[vhipp_pads[j-len(mpfc_pads)]]
        power_corrs[i][j] = pd.Series.corr(siga, sigb)

fig, ax = plt.subplots(figsize=(14,14))
im = ax.imshow(power_corrs, vmin=np.min(power_corrs), vmax=np.max(power_corrs))

ax.set_yticks(np.arange(totlen))
ax.set_xticks(np.arange(totlen))

ax.set_yticklabels(np.concatenate((mpfc_pads, vhipp_pads)))
ax.set_xticklabels(np.concatenate((mpfc_pads, vhipp_pads)))

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(10)
    tick.label.set_rotation('vertical')

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(10)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

fig.tight_layout()
fig.colorbar(im, shrink=0.5)
plt.title('Pearson\'s Correlation between mPFC and vHPC signals', fontsize=16)
plt.ylabel('mPFC')
plt.xlabel('vHPC')
plt.show()

In [ ]:
exclude = [29, 30, 31, 32, 34, 43, 52, 60, 61, 62, 63]

In [ ]:
mpfc_ch_labels, vhipp_ch_labels, FWHMs = plotting.plot_phase_coh_pairs(dataset, animal, session, savedir=savedir+'phase_diff_distrib/', band='theta', exclude=exclude,
                              srate=f_ephys, beh_srate=f_behavior, tstart=start_time, twin=behavior_window_duration, nbins=64,
                             select_idx=None)

In [ ]:
segs = []
pos = behavior_trigger
for i in range(200):
    segs.append(pos)
    pos += 2.5

In [ ]:
from scipy.signal import correlate, correlation_lags

power_mpfc = ephys.column_by_pad(ephys.get_power(dataset, 'mpfc', 'theta'))
power_vhipp = ephys.column_by_pad(ephys.get_power(dataset, 'vhipp', 'theta'))

mpfc_pads = np.array(power_mpfc.columns)
vhipp_pads = np.array(power_vhipp.columns)

timelag_peakpos = []

for mpfc_chid in range(len(mpfc_pads)):
    for vhipp_chid in range(len(vhipp_pads)):
        if (not vhipp_pads[vhipp_chid] in exclude) and (not mpfc_pads[mpfc_chid] in exclude):
            vhipp_mean = np.mean(np.array(power_vhipp[vhipp_pads[vhipp_chid]]))
            pair_lags = []
            validsegs = 0
            for seg in segs:
                segstart = seg / f_behavior
                segend = segstart + 2.5
                crop_from = int(segstart * f_ephys)
                crop_to = int(segend * f_ephys)
                power_mpfc_crop = np.array(power_mpfc[mpfc_pads[mpfc_chid]])[crop_from:crop_to]
                power_vhipp_crop = np.array(power_vhipp[vhipp_pads[vhipp_chid]])[crop_from:crop_to]
                vhipp_mean_crop = np.mean(power_vhipp_crop)
                if vhipp_mean_crop > vhipp_mean:
                    validsegs += 1
                    corr = correlate(power_mpfc_crop, power_vhipp_crop)
                    corr /= np.max(corr)
                    lags = correlation_lags(len(power_mpfc_crop), len(power_vhipp_crop)) / f_ephys * 1000
                    lag = lags[np.argmax(corr)]
                    pair_lags.append(lag)
#             print(validsegs)

            if validsegs > 0:
                pair_lags_all = np.array(pair_lags).flatten()
                plt.figure(figsize=(10,10))
                fig, ax = plt.subplots()
                bin_edges = np.linspace(-1000, 1000, num=1000)
                n, bins, patches = ax.hist(pair_lags_all, bin_edges, histtype='stepfilled')
                peak_value = np.max(n)
                bin_max = np.where(n == peak_value)
                timelag_peakpos.append(bins[bin_max][0])
                ax.tick_params(axis='both', which='major', labelsize=8)
                ax.set_xlabel('Time lag (ms)', labelpad=18, fontsize=12)
                ax.set_ylabel('Counts of 2.5s time segments', labelpad=18, fontsize=12)
                ax.set_title('Time lags of channel pair ' + str(mpfc_pads[mpfc_chid]) + '-' + str(vhipp_pads[vhipp_chid]) + ' across time intervals', fontsize=14)
                plt.savefig(savedir+'pair_lags/'+animal[session]+'_' + str(mpfc_pads[mpfc_chid]) + '-' + str(vhipp_pads[vhipp_chid])+ '_lags.jpg', bbox_inches = 'tight')
                plt.show()
                plt.clf()

In [ ]:
peaks_all = np.array(timelag_peakpos).flatten()
plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
bin_edges = np.linspace(-50, 50, num=50)
n, bins, patches = ax.hist(peaks_all, bin_edges, histtype='stepfilled')

ax.tick_params(axis='both', which='major', labelsize=8)
ax.set_xlabel('Time lag (ms)', labelpad=18, fontsize=12)
ax.set_ylabel('Counts of mPFC-vHPC channel pairs', labelpad=18, fontsize=12)
ax.set_title('Peak vHPC-mPFC time lag occurrance distribution (arena)', fontsize=14)
ax.set_ylim(top=800)
plt.savefig(savedir+animal[session]+'_arena_lags.jpg', bbox_inches = 'tight')